# Exploring Gender Biases in Word2vec <br> 

Write a short intro here 


In [14]:
#Imports
import torch
from torch import nn, optim, sigmoid, softmax
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
import string
import numpy as np
from numpy.linalg import norm

from gutenberg_data import get_urls, read_data_from_urls


## 1- Define Word2Vec Model 

**Create Tokens** <br>

Justifications:

- punctuation: remove - source : ______________
- lowercase words - because we are only looking at biasis between male/female

In [6]:
def create_tokens(data, min_freq):
    """
    create_vocabulary finds the number of occurences of each word, creates vocuabulary with words with > N_freq, and lebels them with ids

    :param data: list of strings
    :return: a list of tokens 
    """ 
    tokens = []
    for sentance in data:
      #split into tokens, convert to lowercase 
      sentance = sentance.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
      sentance = sentance.translate(str.maketrans('', '', string.digits)) #remove digits
      tokenizer = get_tokenizer("basic_english", language="en") #remove unessasary characters, splits into spaces
      tokens.append(tokenizer(sentance))
  
    return tokens
  

**Define word 2 vec model** <br>

- choosing the top N most frequent words - why? so its easier to train - and not wasting time to train words that appear once in a text sequence. 

In [22]:
# get test data so far
train_urls = get_urls('test')
train_data = read_data_from_urls(train_urls)
train_data=str(train_data[0]).split('.')

In [63]:
from gensim.models import Word2Vec
min_freq = 1
size = 300
window = 10

sentences = create_tokens(train_data, min_freq)

model = Word2Vec(min_count=min_freq, vector_size=size, window=window, sg=1) #sg=1 is the skip-gram training algorithm
model.init_weights()
model.build_vocab(sentences)  # prepare the model vocabulary
#model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)  # train word vectors


#Useful helper functions for model

def check_w_embedding(word_lists, model, printRemoved):
    new_w_lists = []
    
    for word_list in word_lists:
        new_w_l = []
        for w in word_list: 
            if w in model.wv.key_to_index:
                new_w_l.append(w)
            else:
                if printRemoved:
                    print ("Word", w, " is not in the model")
                    
        new_w_lists.append(new_w_l)
    
    return new_w_lists
    

def w_vec(word):
    '''
    returns the word embedding of the input word
    '''
    return model.wv[word]
   

def cos_sim(a,b):
    '''
    returns the cosine similarity between 2 word embeddings 
    '''
    return np.dot(a, b)/(norm(a)*norm(b))


#WORD2VEC TUTORIAL (operations):
vector_comp = model.wv['she']
vector_comp_2 = model.wv['he']

sims = model.wv.most_similar('she', topn=10)  # get other similar words
print(sims)

cosine_sim = model.wv.similarity('her', 'she')
print(cosine_sim)


#https://radimrehurek.com/gensim/models/word2vec.html?fbclid=IwAR0yK8_da0wQdA1jCofwIfOR42Kj4mSVvxTsPG4RS8ckai-s6-YvmyrRFRs#gensim.models.word2vec.Word2Vec

[('bondsrncould', 0.2044210433959961), ('endeavoring', 0.2040293961763382), ('travelers', 0.19688072800636292), ('doctorsrnprinters', 0.1957394927740097), ('pioneerrnminnesotian', 0.1870165467262268), ('dunnell', 0.18451200425624847), ('drawn', 0.18440429866313934), ('think', 0.1819811463356018), ('minnehaharnfalls', 0.1775171011686325), ('containing', 0.1764126718044281)]
-0.0068912134


## 2 - Create Data sets with wiki and ArXiv from the Pile

## 3 - Debiasing Techniques


### 3.1 - Zhao et al.'s Method Using an Objective Function with Catigorized Words


### 3.2 - Bolukbasi et al.'s Soft-debaising Method

### 3.3 - Savani et al.'s First Order Optimization

## 4 - Measuring Bias

### 4.1 - Direct Bias

### 4.2 - Indirect Bias


For indirect bias, we can define the gender component of words, β(w, v), by defining wg , the contribution from gender, and w⊥ = w −wg, where the word vectors w are unit normal. The equation for β(w, v) from Bolukbasi et al.  is shown below. 
				(w, v)= (wv - wv||w||2||v||2)/(wv)
Larger β(w, v) suggest similarity of embeddings that have no relation (softball and receptionist) which can be largely explained by gender biases in the embedding. Furthermore, to visualize these similarities, we can use k-means to split the embeddings into clusters and analyze the indirect bias that way, as was done in Gohen et al. 

wg = (w · g)g


In [ ]:
def calculate_indirect_metric(g, w, v):
    wg= np.dot(np.dot(w,g),g)
    vg = np.dot(np.dot(v,g),g)
    w_norm_vec = w-wg
    v_norm_vec =v-vg
    w_norm = norm(w_norm_vec)
    v_norm = norm(v_norm_vec)
    
    return (np.dot(w,v) - np.dot(w_norm_vec,v_norm_vec)/(w_norm*v_norm))/np.dot(w,v)


#need to define w, v words which we want to calulate this metric on (in a loop)
    

### 4.3 - WEAT Metric



- so far only works for google's word2vec model 
- based on the equations: 



In [72]:
def s_word_A_B(w,A,B):
    mean_A_sum = 0
    mean_B_sum = 0
    w_v = w_vec(w)
    
    for a in A:
        a_vec = w_vec(a)
        mean_A_sum +=cos_sim(w_v,a_vec)
    
    for b in B:
        b_vec = w_vec(b)
        mean_B_sum +=cos_sim(w_v,b_vec)
        
    return mean_A_sum/len(A) - mean_B_sum/len(B)

def s_X_Y_A_B(X,Y,A,B):
    sum_X = 0
    sum_Y = 0
    
    for x in X:
        sum_X += s_word_A_B(x,A,B)
        
    for y in Y:
        sum_Y += s_word_A_B(y,A,B)
    
    return sum_X - sum_Y

Now, lets run 3 experiments on the word2vec model

In [73]:

#Experiment 1

#A and B are the attribute word groups 
A = ['tie', 'manager', 'work', 'paper', 'money', 'office', 'business', 'meeting']
B = ['home', 'parents', 'children', 'family', 'sister', 'marriage', 'charm', 'relatives']

#target words 
X = ['he', 'him'] #male 
Y = ['she', 'her'] #female

A,B,X,Y = check_w_embedding([A,B,X,Y], model, True)

metric_exp_1 = s_X_Y_A_B(X,Y,A,B)

print (metric_exp_1)


-0.006268933910178021


## 5 - Comparison and Conclusions